1. Chicago Socioeconomic Indicators

This dataset contains a selection of six socioeconomic indicators of public health significance and a hardship index, by Chicago community area, for the years 2008 – 2012.

2. Chicago Public Schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year.

3. Chicago Crime Data

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days.


In [1]:
import csv, sqlite3

con = sqlite3.connect("RealWorldData.db")
cur = con.cursor()

In [3]:
!pip install pandas


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Dell\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------------- ---------------------- 0.8/1.8 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 4.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1


In [4]:
!pip install ipython-sql

Defaulting to user installation because normal site-packages is not writeable


In [5]:
%load_ext sql

In [6]:
%sql sqlite:///RealWorldData.db

In [8]:
import pandas
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False)

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False)

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False)


NameError: name 'con' is not defined

In [17]:
# Step 3: Set the table display style to one that exists (e.g., 'DEFAULT' replaced with 'DEFAULT' manually)
import prettytable
from sql import config
config.style = 'PLAIN_COLUMNS'  # This avoids the KeyError


ImportError: cannot import name 'config' from 'sql' (C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sql\__init__.py)

In [16]:
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///RealWorldData.db
Done.


KeyError: 'DEFAULT'

Query the database system catalog to retrieve column metadata

In [7]:
%sql SELECT count(name) FROM PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS_DATA');

 * sqlite:///RealWorldData.db
Done.


count(name)
78


Problem 1: Find the total number of crimes recorded in the CRIME table.

In [8]:
# Rows in Crime Table
%sql select COUNT(*) AS TOTAL_CRIMES \
from CHICAGO_CRIME_DATA

 * sqlite:///RealWorldData.db
Done.


TOTAL_CRIMES
533


Problem 2: List community areas with per capita income less than 11000.

In [9]:
%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME
West Garfield Park
South Lawndale
Fuller Park
Riverdale


Problem 3: List all case numbers for crimes involving minors?

In [10]:
%sql SELECT DISTINCT CASE_NUMBER FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%MINOR%'

 * sqlite:///RealWorldData.db
Done.


CASE_NUMBER
HL266884
HK238408


Problem 4: List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)

In [11]:
%sql SELECT DISTINCT CASE_NUMBER, PRIMARY_TYPE, DATE, DESCRIPTION FROM CHICAGO_CRIME_DATA \
WHERE PRIMARY_TYPE='KIDNAPPING'

 * sqlite:///RealWorldData.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DATE,DESCRIPTION
HN144152,KIDNAPPING,2007-01-26,CHILD ABDUCTION/STRANGER



Problem 5: What kind of crimes were recorded at schools?

In [12]:
%sql SELECT DISTINCT(PRIMARY_TYPE), LOCATION_DESCRIPTION FROM CHICAGO_CRIME_DATA \
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///RealWorldData.db
Done.


PRIMARY_TYPE,LOCATION_DESCRIPTION
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"
PUBLIC PEACE VIOLATION,"SCHOOL, PRIVATE, BUILDING"
PUBLIC PEACE VIOLATION,"SCHOOL, PUBLIC, BUILDING"


Problem 6: List the average safety score for all types of schools.

In [13]:
%sql SELECT "Elementary, Middle, or High School", AVG(SAFETY_SCORE) AVERAGE_SAFETY_SCORE \
FROM CHICAGO_PUBLIC_SCHOOLS_DATA GROUP BY "Elementary, Middle, or High School";

 * sqlite:///RealWorldData.db
Done.


"Elementary, Middle, or High School",AVERAGE_SAFETY_SCORE
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


Problem 7: List 5 community areas with highest % of households below poverty line.

In [14]:
%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5 ;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


Problem 8: Which community area(number) is most crime prone?

In [15]:
%%sql
SELECT COMMUNITY_AREA_NUMBER ,COUNT(COMMUNITY_AREA_NUMBER) AS FREQUENCY
FROM CHICAGO_CRIME_DATA 
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
LIMIT 1;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NUMBER,FREQUENCY
25.0,43


Problem 9: Use a sub-query to find the name of the community area with highest hardship index.

In [16]:
%sql SELECT COMMUNITY_AREA_NAME FROM  CENSUS_DATA WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA);

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME
Riverdale


Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [17]:
%%sql
SELECT community_area_name FROM CENSUS_DATA 
WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER FROM CHICAGO_CRIME_DATA 
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
    LIMIT 1)
LIMIT 1;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME
Austin
